In [1]:
import numpy as np
import matplotlib.pylab as pl
import ot
import ot.plot
import numpy.random
from numpy import random
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage.filters import laplace

ModuleNotFoundError: No module named 'ot'

# Convolutional Sinkhorn

In [6]:
#%% parameters and data generation

n = 100  # nb samples

mu_n = np.array([random.uniform(0,3) for x in range(n)])

nu_n_init = [random.uniform(0,1) for x in range(n)]

def nu_sample(unif):
    if unif < 1/4:
        return unif**(1/2)
    if unif >= 1/4 and unif < 1/2:
        return 1-(1/2-unif)**(1/2)
    if unif >= 1/2 and unif < 3/4:
        return 2+(unif-1/2)**(1/2)
    if unif >= 3/4:
        return 3-(1-unif)**(1/2)
    
nu_n = np.array(list(map(nu_sample, nu_n_init)))
beta = [10,5,1,0.1,0.001]

# a, b = np.ones((n,)) / n, np.ones((n,)) / n 

# #loss matrix
# M = ot.dist(mu_n.reshape((n,1)), nu_n.reshape((n,1)), metric="euclidean")
# M /= M.max()

In [72]:
def convolutional_wasserstein(mu0, mu1, gamma, threshold = 1e-15, iteration = 10000, base=1e-30):
    m = list(set(list(mu0) + list(mu1)))
    mu00 = pd.Series([1/len(mu0)]*len(mu0)+[0]*len(mu1), index=list(mu0)+list(mu1)).values
    # print(mu0)
    mu11 = pd.Series([0]*len(mu0)+[1/len(mu1)]*len(mu1), index=list(mu1)+list(mu0)).values
    a = np.ones_like(mu00)
    v = np.ones_like(mu00)
    w = np.ones_like(mu00)
    obj = 100
    j = 0
    # L = laplace(m)
    def Ht(x=a*w,t=gamma/2):
        xy = np.linspace(0, 3, mu00.shape[0])
        [Y, X] = np.meshgrid(xy, xy)
        ht = np.exp(-(X-Y) ** 2/4/t)
        return [np.dot(np.dot(ht, x), ht), ht]
    # ht = np.linalg.inv(np.diag(a)+gamma/2*L)
    while (obj > threshold and j < iteration) :
        v0 = v
        w0 = w
        v = mu00/np.maximum(base, Ht(a*w)[0])
        w = mu11/np.maximum(base, Ht(a*v)[0])
        j += 1
        obj = sum((v-v0)**2+(w-w0)**2)
    # print(np.sum(np.dot(np.dot(np.diag(v),Ht()[1]),np.diag(w))))
    # , 'pi': np.dot(np.dot(np.diag(v),Ht()[1]),np.diag(w))
    return {'loss':gamma*np.dot(a, mu00*np.log(np.where(v==0, 1, v))+mu11*np.log(np.where(w==0, 1, w))), 'step': j}

In [75]:
otarray = convolutional_wasserstein(mu_n, nu_n, beta[-1])
otarray

{'loss': 0.10311769399171576, 'step': 1590}